In [ ]:
"""
premier jet pour automatisation des plots pour comparaison et tout MAIS pbm avec hvplot dans une boucle
"""
# comparison.ipynb

# This programme  

# C. Mackay March 2023 (Catherine.Mackay@aero.obs-mip.fr)
# https://github.com/ckmackay/SOFT-IO-LI.git

#Suggestions/improvements to be made:


In [14]:
# IMPORTS
import os
import os.path
from os.path import exists
from netCDF4 import Dataset

#from ncdump import ncdump
import numpy as np
import pandas as pd
import xarray as xr
import json

#for plotting
import hvplot.xarray # fancy plotting for xarray
import holoviews as hv
import matplotlib.pyplot as plt
import geoviews as gv

#for stats
import statistics
from statistics import mean, median, mode, stdev, median_high

#for datetime
import datetime
from datetime import datetime as dt
import matplotlib.dates as mdates
from matplotlib.dates import date2num

#for cartopy
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature

In [7]:
### Check the following paths and settings: these are set for SOFT-IO-LI default on nuwa in /o3p/iagos/SOFT-IO-LI
### For testing, think of creating /o3p/user/SOFT-IO-LI/ and changing the paths below. 
### you will need to create the following directories:

def set_paths():
    
    global flexpart_path, flight_path
    
    #flexpart_path = '/o3p/macc/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/SOFT-IO-LI/2015/05/'
    flexpart_path = '/o3p/macc/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/'
    flight_path = '/o3p/iagos/iagosv2/netcdf/L2/'
    

In [8]:
# find flights from directory

def find_flights():
    global flights
    flights=[]
    flights=(os.listdir(flexpart_path))
    #print(flights)
    #print(len(flights))

In [10]:
# open netcdf file for flight

def open_netcdf():
    global da, netcdf_file
    i=0
    for file in os.listdir(flexpart_path+flights[i]+'/10j_100k_output/'):
        if file.endswith(".nc"):
            #print(file)
            
            ds = xr.open_dataset(flexpart_path+flights[i]+'/10j_100k_output/'+file)
            ds.spec001_mr.sizes
            da = ds.spec001_mr.chunk({'nageclass': 1, 'pointspec': 1, 'time': 40, 'height': -1, 'latitude': -1, 'longitude': -1})
            # let's get rid of the auxilary dimension "nageclass", for the sake of clarity:
            da = da.squeeze(dim='nageclass')
            spec_argmax = da.argmax(dim=['longitude', 'latitude'])
            # let's grab longitudes and latitudes of max. value of spec001_mr:
            lon_argmax = spec_argmax['longitude']
            lat_argmax = spec_argmax['latitude']
            # let's see what's in lon_argmax, for example
            #lon_argmax
            # the data is not yet computed; in order to actually fire the computation (load data from netcdf and process it), we can call a method "load"; it will take a couple of seconds to execute:
            lon_argmax.load()
            lat_argmax.load()
            # and let's see what's in lon_argmax, for example:
            #lon_argmax
            spec_max = da.isel(longitude=lon_argmax, latitude=lat_argmax)
            # again, we have to actually load spec_max to see results:
            spec_max.load()
            lon_argmax = lon_argmax.where(spec_max > 0)
            lat_argmax = lat_argmax.where(spec_max > 0)
            spec_max = spec_max.where(spec_max > 0)
            #spec_max

            da = da.assign_coords(pointspec=da.pointspec)
         
    return(da)

In [11]:
# set airport coords

def set_airport_coords():
    global san_diego,frankfurt, bogota, cologne
    san_diego = hv.Scatter((-117.1933, 32.7338)).opts(color='pink', size=3)
    frankfurt = hv.Scatter((8.5622, 50.0379)).opts(color='pink', size=3)
    bogota = hv.Scatter((-74.1461, 4.7010)).opts(color='pink', size=3)
    cologne = hv.Scatter((6.9603, 50.9375)).opts(color='pink', size=3)
    

In [24]:
def plot_all(da):
    global pl
    da_all_agregated = da.isel(pointspec=0).sum(dim=('height', 'time'))
    pl = da_all_agregated.hvplot.image(
        #x='longitude', y='latitude',  title='Max residence time for all times and altitudes',
        x='longitude', y='latitude',  title='Plume dispersion over 10 days',
        xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
        frame_height=350, 
        projection=ccrs.PlateCarree(), 
        geo=True, project=False, global_extent=True,
        framewise=False, coastline=True, dynamic=True,
        cmap='coolwarm',
    ).opts(fontsize={'title': 16, 'xlabel': 20, 'xticks': 6, 'yticks': 12, 'legend': 12})
    pl
    #* san_diego * frankfurt * flight_path * first_rel
    return(pl)

In [ ]:
def flight_path(da):
    global flight_path
    
    #sort out finding the correct IAGOS_timeseries netCDF file automatically
    
    db = xr.open_dataset('/o3p/macc/test/NOx_synched/NOx_IAGOS_timeseries_2018060508235702.nc')
    flight_path = hv.Scatter((db.lon, db.lat)).opts(color='red', size=0.25, )
    
    

In [25]:
# main programme
set_paths()
find_flights()
set_airport_coords()
print("flight path = ", flight_path)
#for i in range(len(flights)):
for i in range(0,1):
    open_netcdf()
    
    flight =(flights[i])
    print(flight)
    
    flight_path()
    """
    - pbm pour plots dans boucle, peut-être si on arrive à nommer les graphiques et ensuite on ajoute les overlays plus tard
    peut-être donner autre chose que da
    """
    plot_all(da)
    #extract_plume_info(flight, flight_path)
    #print("extract plume completed")
    #verify_max_res_time_loc(da)
    

flight path =  /o3p/iagos/iagosv2/netcdf/L2/
flight_2018_014_1h_05deg


In [26]:
pl

:Overlay
   .Image.I     :Image   [longitude,latitude]   (spec001_mr)
   .Coastline.I :Feature   [Longitude,Latitude]

In [30]:
pl * san_diego * frankfurt

:Overlay
   .Image.I     :Image   [longitude,latitude]   (spec001_mr)
   .Coastline.I :Feature   [Longitude,Latitude]
   .Scatter.I   :Scatter   [x]   (y)
   .Scatter.II  :Scatter   [x]   (y)

In [31]:
pl * san_diego * frankfurt * flight_path

TypeError: can't multiply sequence by non-int of type 'Overlay'